In [3]:
import pandas as pd
import re
import copy

In [4]:
forum_messages_df = pd.read_csv('../data/ForumMessages.csv')
forum_messages_df.head()
#print(forum_messages_df['Message'][:2][0])
#print(forum_messages_df['Message'])

,Id,ForumTopicId,PostUserId,PostDate,ReplyToForumMessageId,Message,Medal,MedalAwardDate
0,667078,115913,1788308,11/06/2019 19:40:03,666530.0,"<p><a href=""/manojprabhaakr"">@manojprabhaakr</...",NaN,NaN
1,667077,115913,1788308,11/06/2019 19:38:55,666668.0,"<p><a href=""/cdeotte"">@cdeotte</a> </p>\n\n<p>...",3.0,11/06/2019
2,667076,74968,3961461,11/06/2019 19:38:19,NaN,"<p>A very detailed and helpful notebook, \nTha...",NaN,NaN
3,667075,115817,1666986,11/06/2019 19:37:59,NaN,<p>You don't say. You might just got your wish...,NaN,NaN
4,667074,113468,1073620,11/06/2019 19:34:36,666591.0,"<p>Hi <a href=""/mobassir"">@mobassir</a> If I ...",3.0,11/07/2019


In [5]:
wikipedia_movie_plots_df = pd.read_csv('../data/wiki_movie_plots_deduped.csv')
wikipedia_movie_plots_df.head()
#print(wikipedia_movie_plots_df['Plot'])

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [6]:
emotions_db_df = pd.read_csv('../data/emotions_db_text.csv')
emotions_db_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
# Function to remove HTML tags from a string
def remove_html_tags(text):
    if isinstance(text, str):
        # Regular expression for finding HTML tags
        clean = re.compile('<.*?>')
        # Replacing the tags with an empty string
        return re.sub(clean, '', text)
    else:
        return ''

forum_messages_df['Message'] = forum_messages_df['Message'].apply(lambda x: remove_html_tags(x))

forum_messages_df.head()

In [7]:
original_texts = pd.concat([forum_messages_df['Message'],wikipedia_movie_plots_df['Plot']], ignore_index=True)
original_texts_wiki_plot = copy.deepcopy(wikipedia_movie_plots_df['Plot'])
original_texts_wiki_plot

0        A bartender is working at a saloon, serving dr...
1        The moon, painted with a smiling face hangs ov...
2        The film, just over a minute long, is composed...
3        Lasting just 61 seconds and consisting of two ...
4        The earliest known adaptation of the classic f...
                               ...                        
34881    The film begins in 1919, just after World War ...
34882    Two musicians, Salih and Gürkan, described the...
34883    Zafer, a sailor living with his mother Döndü i...
34884    The film centres around a young woman named Am...
34885    The writer Orhan Şahin returns to İstanbul aft...
Name: Plot, Length: 34886, dtype: object

In [8]:
emotions_db_text_df = copy.deepcopy(emotions_db_df['text'])
emotions_db_text_df[:5]

0        i just feel really helpless and heavy hearted
1    ive enjoyed being able to slouch about relax a...
2    i gave up my internship with the dmrg and am f...
3                           i dont know i feel so lost
4    i am a kindergarten teacher and i am thoroughl...
Name: text, dtype: object

In [9]:
rewrite_prompts = [
    'Explain this to me like I\'m five.',
    'Convert this into a sea shanty.',
    'Make this rhyme.',
    'Make this shorter.',
    'Make this longer.',
    'Make this more detailed.',
    'Rewrite this essay but do it using the writing style of Dr. Seuss',
    'Rewrite this essay but do it using the writing style of William Shakespeare',
    'Rewrite this essay but do it using the writing style of Tupac Shakur',
    'Make this a haiku.',
    'Make this into a poem.',
    'Turn this into a sonnet.',
    'Summarize this.',
    'Give me the highlights.',
    'Rewrite this essay using the writing style of Jane Austen.',
    'Rewrite this essay in the writing style of Ernest Hemingway.',
    'Transform this piece to reflect Virginia Woolf\'s writing style.',
    'Rephrase this essay using Mark Twain\'s distinctive writing style',
    'Revise this text to mirror J.K. Rowling\'s writings.',
    'Recreate this content with George Orwell\'s writing.',
    'Convert this text into a form that evokes Edgar Allan Poe\'s gothic style, focusing on the macabre, the mysterious, and the psychological depth of his narratives.'
]

In [ ]:
!pip install -q -U immutabledict sentencepiece
!git clone https://github.com/google/gemma_pytorch.git

In [10]:
import sys
sys.path.append("gemma_pytorch")
from gemma_pytorch.gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM
from gemma_pytorch.gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

VARIANT = "7b-it-quant"
MACHINE_TYPE = "cuda"
weights_dir = '../models/gemma-7b-it'

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)
    
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")
model_config.quant = "quant" in VARIANT

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()

In [11]:
from tqdm import tqdm
import random
random.seed(0)
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

rewrite_data = []

for original_text in tqdm(emotions_db_text_df[5:50]):
    rewrite_prompt = random.choice(rewrite_prompts)
    prompt = f'{rewrite_prompt}\n{original_text}'
    rewritten_text = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=100,
    )
    rewrite_data.append({
        'original_text': original_text,
        'rewrite_prompt': rewrite_prompt,
        'rewritten_text': rewritten_text,
    })

  0%|          | 0/45 [05:00<?, ?it/s]


KeyboardInterrupt: 

In [15]:
rewrite_data_df = pd.DataFrame(rewrite_data)
#rewrite_data_df[:20].values

In [16]:
rewrite_data_df.to_csv('prompts_and_rewrites_n.csv', index=False)